
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# エンコーダ/デコーダのアーキテクチャ

:label: `sec_encoder-decoder`

機械翻訳 (:numref: `sec_machine_translation` ) のような一般的な seq2seq 問題では、入力と出力は整列されていないさまざまな長さになります。この種のデータを処理する標準的なアプローチは、可変長シーケンスを入力として受け取る*エンコーダー*と、条件付き言語として機能する*デコーダー*という 2 つの主要コンポーネントで構成される*エンコーダー デコーダー*アーキテクチャ (:numref: `fig_encoder_decoder` ) を設計することです。モデルでは、エンコードされた入力とターゲット シーケンスの左方向のコンテキストを取得し、ターゲット シーケンス内の後続のトークンを予測します。 

![](../img/encoder-decoder.svg):label: `fig_encoder_decoder`

例として、英語からフランス語への機械翻訳を見てみましょう。英語の入力シーケンス「They」、「are」、「watching」、「.」が与えられると、このエンコーダー/デコーダー アーキテクチャは、まず可変長入力を状態にエンコードし、次にその状態をデコードして、翻訳されたシーケンス、トークンを生成します。出力としてトークンごと: "Ils"、"regardent"、"."。エンコーダ/デコーダ アーキテクチャは後続のセクションでさまざまな seq2seq モデルの基礎を形成するため、このセクションではこのアーキテクチャを後で実装されるインターフェイスに変換します。


In [1]:
from torch import nn
from d2l import torch as d2l


## (**エンコーダ**)

エンコーダー インターフェイスでは、エンコーダーが可変長シーケンスを入力`X`として受け取ることを指定するだけです。実装は、この基本`Encoder`クラスを継承する任意のモデルによって提供されます。


In [2]:
class Encoder(nn.Module):  #@save
    """The base encoder interface for the encoder-decoder architecture."""
    def __init__(self):
        super().__init__()

    # Later there can be additional arguments (e.g., length excluding padding)
    def forward(self, X, *args):
        raise NotImplementedError


## [**デコーダ**]

次のデコーダ インターフェイスでは、エンコーダ出力 ( `enc_all_outputs` ) をエンコードされた状態に変換するための`init_state`メソッドを追加します。このステップでは、 :numref: `sec_machine_translation`で説明した入力の有効な長さなど、追加の入力が必要になる場合があることに注意してください。可変長シーケンストークンをトークンごとに生成するには、デコーダが入力 (たとえば、前のタイムステップで生成されたトークン) とエンコードされた状態を現在のタイムステップでの出力トークンにマッピングするたびに行います。


In [3]:
class Decoder(nn.Module):  #@save
    """The base decoder interface for the encoder-decoder architecture."""
    def __init__(self):
        super().__init__()

    # Later there can be additional arguments (e.g., length excluding padding)
    def init_state(self, enc_all_outputs, *args):
        raise NotImplementedError

    def forward(self, X, state):
        raise NotImplementedError


## [**エンコーダとデコーダを組み合わせる**]

順方向伝播では、エンコーダの出力はエンコードされた状態を生成するために使用され、この状態はデコーダによってその入力の 1 つとしてさらに使用されます。


In [4]:
class EncoderDecoder(d2l.Classifier):  #@save
    """The base class for the encoder-decoder architecture."""
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_all_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_all_outputs, *args)
        # Return decoder output only
        return self.decoder(dec_X, dec_state)[0]


次のセクションでは、このエンコーダ/デコーダ アーキテクチャに基づいて RNN を適用して seq2seq モデルを設計する方法を見ていきます。

## まとめ

エンコーダ/デコーダ アーキテクチャは、両方とも可変長シーケンスで構成される入力と出力を処理できるため、機械翻訳などの seq2seq 問題に適しています。エンコーダーは可変長シーケンスを入力として受け取り、それを固定形状の状態に変換します。デコーダは、固定形状の符号化状態を可変長シーケンスにマッピングします。

## 演習
1. ニューラル ネットワークを使用してエンコーダ/デコーダ アーキテクチャを実装すると仮定します。エンコーダーとデコーダーは同じタイプのニューラル ネットワークである必要がありますか?
1. 機械翻訳以外に、エンコーダ/デコーダ アーキテクチャを適用できる別のアプリケーションを思いつきますか?



[ディスカッション](https://discuss.d2l.ai/t/1061)
